In [1]:
import torch 
from torch import nn
from d2l import torch as d2l

In [7]:
def corr2d(X,K):
    """二维互相关运算"""
    h,w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1,X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i + h,j:j+w]*K).sum()
            #这里的乘法是对应位置的乘法，不是矩阵的乘法
    return Y

In [8]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [11]:
#卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        # nn.Parameter：PyTorch 中用于标记 “可学习参数” 的类。
        # 被 nn.Parameter 包装的张量会被自动纳入模型的参数列表（通过 model.parameters() 可访问），
        # 并在训练时参与梯度下降更新。
        self.bias = nn.Parameter(torch.zeros(1))
        
        #torch.zeros(1)：生成一个值为 0 的标量张量（形状为 (1,)），作为偏置的初始值。
        #PyTorch 的广播机制遵循 **“后缘维度匹配”** 原则，简单来说：当两个张量形状不同时，从最后一个维度开始比较，
        # 若维度大小相等，或其中一个为 1，则可以广播；否则无法广播，会报错。
        
        #由于两个张量维度数不同（1 维 vs 2 维），先给维度少的张量左侧补 “长度为 1 的维度”，使二者维度数一致。
        # 偏置 b 补维后：形状从 (1) → (1, 1)（2 维张量，第一维长度 1，第二维长度 1）。
        # 输出特征图 Y：形状保持 (2, 2)（2 维张量，第一维长度 2，第二维长度 2）。
        # 此时两个张量的形状分别为 (1,1) 和 (2,2)，逐维度对比：
        # 第一维：1 vs 2 → 有一个为 1，满足广播条件。
        # 第二维：1 vs 2 → 有一个为 1，满足广播条件。

    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

In [15]:
#图像中目标的边缘检测
#中间4列为黑色(0),其余像素为白色(1)
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [16]:
#卷积核
#在进行互相关运算时,如果水平相邻两元素相同,则输出0,否则输出为非零
K = torch.tensor([[1.0,-1.0]])

In [17]:
Y= corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [20]:
corr2d(X.t(),K)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [25]:
#学习卷积核
#构造一个二维卷积层,它具有一个输出通道和形状为(1,2)的卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias = False)
# nn.Conv2d：PyTorch 内置的二维卷积层类。
# 参数说明：
# 1（第一个参数）：in_channels=1，输入通道数为 1（比如灰度图像）。
# 1（第二个参数）：out_channels=1，输出通道数为 1（只学习 1 个卷积核）。
# kernel_size=(1, 2)：卷积核形状为 1 行 2 列（横向的 1x2 核，适合检测水平方向的变化）。
# bias=False：不使用偏置项（简化学习过程）。
# 此时卷积核的初始值是随机的（PyTorch 默认初始化），后续通过训练更新


#这个二维卷积层使用四维输入和输出格式(批量大小,通道,高度,宽度)
#其中批量大小和通道数都为1
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
#PyTorch 的 nn.Conv2d 是为批量处理多通道数据设计的，它要求输入必须是四维张量，形状严格遵循：(batch_size, in_channels, height, width)四个维度的含义分别是：
# batch_size：批量大小（一次输入的样本数量）。,图像个数
# in_channels：输入通道数（如灰度图为 1，RGB 图为 3）。
# height：输入特征图的高度（行数）。
# width：输入特征图的宽度（列数）

lr = 3e-2 #学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat -Y)**2
    conv2d.zero_grad()# 清零梯度：避免上一轮的梯度累积
    l.sum().backward()# 反向传播：计算损失对卷积核的梯度
    #迭代卷积核
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'epoch{i+1},loss{l.sum():.3f}')

epoch2,loss10.951
epoch4,loss2.873
epoch6,loss0.906
epoch8,loss0.326
epoch10,loss0.126


In [26]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9507, -1.0224]])